# Visualize the slab and the earthquake events

In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import vis_pkg
import plotly
import plotly.graph_objects as go
from plotly.offline import plot

# Define functions to use

In [81]:
def rotate_vec(aor_vec, vec, theta):
    
    """
    Rotate vec by angle theta around aor_vec
    All vectors are in cartesian coordinates and written as (x, y, z)
    Theta should already be following the right-hand rule and in radians already
    Function will normalize the aor_vec and vec and will convert theta to radians
    """
    
    theta_rad = np.pi*theta/180
    
    # normalize the input vectors 
    aor_vec_mag = np.sqrt(np.dot(aor_vec, aor_vec))
    norm_aor = aor_vec/aor_vec_mag
    vec_mag = np.sqrt(np.dot(vec, vec))
    norm_vec = vec/vec_mag
    
    # to make life easy
    kx = norm_aor[0]
    ky = norm_aor[1]
    kz = norm_aor[2]
    
    k_mat = np.array([[0, -kz, ky], [kz, 0, -kx], [-ky, kx, 0]])
    
    r_mat = np.identity(3) + np.sin(theta_rad)*k_mat + (1-np.cos(theta_rad))*np.matmul(k_mat, k_mat)
    
    rot_vec = vec_mag*np.matmul(r_mat, norm_vec)
    
    return rot_vec

# Read the needed data

In [82]:
eq_fname = '/Users/jgra0019/Documents/phd_data/focal_meca/ISC_data/clean_isc_gcmt_data.csv'
slab_xyz = '/Users/jgra0019/Documents/phd_data/sumatra_slab/sum_slab2_dep_02.23.18.xyz'
trench_xy = '/Users/jgra0019/Documents/phd_data/trench/sumatra_trench_inv.txt'


# need to be careful with the data
eq_df = pd.read_csv(eq_fname)
trench_df = pd.read_csv(trench_xy, sep=',', header = None)
slab_df = pd.read_csv(slab_xyz, header = None)
slab_df = slab_df[~slab_df[2].isnull()] # remove null values

trench_df.columns = ['LON', 'LAT']
slab_df.columns = ['LON', 'LAT', 'DEPTH']    

In [83]:
slab_df['DEPTH'].max()

-3.1468403339399997

# Plot the slab, trench and events
- Convert slab, trench, events to cartesian
- Plot slab as a surface: done
- Plot trench as a line: done
- Plot events as points: done
- Plot events as lines showing the tension and compression axes

### Step 1: convert data to cartesian

In [84]:
r_earth = 6371 # in km

# slab dataframe
slab_df['DEPTH'] = r_earth + slab_df['DEPTH']
slab_df['LON'] = np.pi*slab_df['LON']/180
slab_df['LAT'] = np.pi*slab_df['LAT']/180

slab_df['X'] = slab_df['DEPTH']*np.cos(slab_df['LON'])*np.sin(0.5*np.pi - slab_df['LAT'])
slab_df['Y'] = slab_df['DEPTH']*np.sin(slab_df['LON'])*np.sin(0.5*np.pi - slab_df['LAT'])
slab_df['Z'] = slab_df['DEPTH']*np.cos(0.5*np.pi - slab_df['LAT'])

# trench data frame
trench_df['DEPTH'] = r_earth
trench_df['LON'] = np.pi*trench_df['LON']/180
trench_df['LAT'] = np.pi*trench_df['LAT']/180

trench_df['X'] = trench_df['DEPTH']*np.cos(trench_df['LON'])*np.sin(0.5*np.pi - trench_df['LAT'])
trench_df['Y'] = trench_df['DEPTH']*np.sin(trench_df['LON'])*np.sin(0.5*np.pi - trench_df['LAT'])
trench_df['Z'] = trench_df['DEPTH']*np.cos(0.5*np.pi - trench_df['LAT'])

# earthquake data frame
eq_df['DEPTH'] = r_earth - eq_df['DEPTH']
eq_df['LON'] = np.pi*eq_df['LON']/180
eq_df['LAT'] = np.pi*eq_df['LAT']/180

eq_df['X'] = eq_df['DEPTH']*np.cos(eq_df['LON'])*np.sin(0.5*np.pi - eq_df['LAT'])
eq_df['Y'] = eq_df['DEPTH']*np.sin(eq_df['LON'])*np.sin(0.5*np.pi - eq_df['LAT'])
eq_df['Z'] = eq_df['DEPTH']*np.cos(0.5*np.pi - eq_df['LAT'])


### Step 2: Solve for the tension and compression axes for plotting and analysis
1. Create axis of rotation 1 (aor1_vec) vectors - which are perpendicular to surface
2. Create north vectors (n_vec)
3. Rotate nvec w.r.t. aor1_vec by amount -azimuth angle: produces az_vec
4. Create axis of rotation 2 (aor2_vec) vectors by rotating the north vectors w.r.t to aor1_vec towards -azimuth + 90 
5. Create final vector by rotating az_vec w.r.t. aor2_vec by plunge angle

In [85]:
eq_df.shape

(3052, 36)

In [86]:
# step 1 - only add to latitude
d_lat = 0.5

dummy_vec = np.zeros([eq_df.shape[0], 3])
dummy_vec[:, 0] = eq_df['DEPTH']*np.cos(eq_df['LON'])*np.sin(0.5*np.pi - (eq_df['LAT'] + np.pi*d_lat/180))
dummy_vec[:, 1] = eq_df['DEPTH']*np.sin(eq_df['LON'])*np.sin(0.5*np.pi - (eq_df['LAT'] + np.pi*d_lat/180))
dummy_vec[:, 2] = eq_df['DEPTH']*np.cos(0.5*np.pi - (eq_df['LAT'] + np.pi*d_lat/180))

aor1_vec = np.zeros([eq_df.shape[0], 3])
aor1_vec[:, 0] = np.array(eq_df['X'] - 0) # zeros are just to make this easy to remember the process
aor1_vec[:, 1] = np.array(eq_df['Y'] - 0)
aor1_vec[:, 2] = np.array(eq_df['Z'] - 0)
mag = np.sqrt(aor1_vec[:, 0]**2 + aor1_vec[:, 1]**2 + aor1_vec[:, 2]**2)
mag = np.reshape(np.repeat(mag, 3), [mag.shape[0], 3])
aor1_vec /= mag

# step 2: create north vectors
n_vec = np.zeros([eq_df.shape[0], 3])
n_vec[:, 0] = np.array(dummy_vec[:, 0] - eq_df['X'])
n_vec[:, 0] = np.array(dummy_vec[:, 1] - eq_df['Y'])
n_vec[:, 0] = np.array(dummy_vec[:, 2] - eq_df['Z'])
mag = np.sqrt(n_vec[:, 0]**2 + n_vec[:, 1]**2 + n_vec[:, 2]**2)
mag = np.reshape(np.repeat(mag, 3), [mag.shape[0], 3])
n_vec /= mag

# step 3: rotate north vectors w.r.t aor1_vec by amount -azimuth angle: produces az_vec
t_az_vec      = np.zeros([eq_df.shape[0], 3]) # 0 - x axis; 1 - y axis; 2 - z axis
t_aor2_vec    = np.zeros([eq_df.shape[0], 3])

p_az_vec      = np.zeros([eq_df.shape[0], 3]) # 0 - x axis; 1 - y axis; 2 - z axis
p_aor2_vec    = np.zeros([eq_df.shape[0], 3])

tension     = np.zeros([eq_df.shape[0], 3])
compression = np.zeros([eq_df.shape[0], 3])

for idx, row in eq_df.iterrows():
    
    ##### work with tension #####
    t_az_vec[idx, :] = rotate_vec(aor1_vec[idx, :], n_vec[idx, :], -row['T_AZM'])
        
    # step 4: create t_aor2_vec vectors by rotating the north vectors w.r.t to aor1_vec towards -azimuth + 90 
    t_aor2_vec[idx, :] = rotate_vec(aor1_vec[idx, :], n_vec[idx, :], -row['T_AZM'] + 90)
                
    # Step 5: create final vector by rotating t_az_vec w.r.t. t_aor2_vec by plunge angle
    tension[idx, :] = rotate_vec(t_aor2_vec[idx, :], t_az_vec[idx, :], row['T_PL'])
    
    ##### work with compression #####
    p_az_vec[idx, :] = rotate_vec(aor1_vec[idx, :], n_vec[idx, :], -row['P_AZM'])
        
    # step 4: create p_aor2_vec vectors by rotating the north vectors w.r.t to aor1_vec towards -azimuth + 90 
    p_aor2_vec[idx, :] = rotate_vec(aor1_vec[idx, :], n_vec[idx, :], -row['P_AZM'] + 90)
                
    # Step 5: create final vector by rotating t_az_vec w.r.t. t_aor2_vec by plunge angle
    compression[idx, :] = rotate_vec(p_aor2_vec[idx, :], p_az_vec[idx, :], row['P_PL'])
    

rot_data = []
for idx, row in eq_df.iterrows():
    
    rot_vec = go.Scatter3d(x = [row['X'] - 100*tension[idx, 0], row['X'], row['X'] + 100*tension[idx, 0]],
                           y = [row['Y'] - 100*tension[idx, 1], row['Y'], row['Y'] + 100*tension[idx, 1]],
                           z = [row['Z'] - 100*tension[idx, 2], row['Z'], row['Z'] + 100*tension[idx, 2]],
                           marker = dict( size = 1,
                                          color = "rgb(84,48,5)"),
                           line = dict( color = "rgb(0,255,0)", # tension in green
                                        width = 6)
                         )
    rot_data.append(rot_vec)
    rot_vec = go.Scatter3d(x = [row['X'] - 100*compression[idx, 0], row['X'], row['X'] + 100*compression[idx, 0]],
                           y = [row['Y'] - 100*compression[idx, 1], row['Y'], row['Y'] + 100*compression[idx, 1]],
                           z = [row['Z'] - 100*compression[idx, 2], row['Z'], row['Z'] + 100*compression[idx, 2]],
                           marker = dict( size = 1,
                                          color = "rgb(84,48,5)"),
                           line = dict( color = "rgb(255,0,0)", # compression in red 
                                        width = 6)
                         )
    rot_data.append(rot_vec)

In [66]:
mag = np.sqrt(n_vec[:, 0]**2 + n_vec[:, 1]**2 + n_vec[:, 2]**2)
print(mag.min())
print(mag.max())

1.0
1.0


In [39]:
# rotate north vectors wrt to aor1 vectors by (-)azimuth angle => out_vec
# create aor2 vectors by rotating the north vectors wrt to aor1 towards (-)azimuth - 90 deg
# create final compression/tension vector by rotating out_vec w.r.t. to aor2 by plunge angle
eq_df.columns

Index(['EVENT_ID', 'DATE', 'TIME', 'LAT', 'LON', 'DEPTH', 'EX', 'MO', 'MW',
       'EX.1', 'MRR', 'MTT', 'MPP', 'MRT', 'MTP', 'MPR', 'STRIKE', 'DIP',
       'RAKE', 'STRIKE.1', 'DIP.1', 'RAKE.1', 'EX.2', 'T_VAL', 'T_PL', 'T_AZM',
       'P_VAL', 'P_PL', 'P_AZM', 'N_VAL', 'N_PL', 'N_AZM', 'X', 'Y', 'Z'],
      dtype='object')

In [87]:

trench_data = go.Scatter3d(
    x = trench_df['X'], 
    y = trench_df['Y'], 
    z = trench_df['Z'], 
    mode = 'markers', 
    marker = dict(size = 1, color = 'black'),
    line=dict(color='black',width=5)
)

# slab_data = go.Scatter3d(
#     x = slab_df.iloc[::10, :]['X'], 
#     y = slab_df.iloc[::10, :]['Y'], 
#     z = slab_df.iloc[::10, :]['Z'], 
#     mode = 'markers', 
#     marker = dict(size = 2, color = 'gray', colorscale = 'Portland')
# )

# eq_data = go.Scatter3d(
#     x = eq_df['X'], 
#     y = eq_df['Y'], 
#     z = eq_df['Z'], 
#     mode = 'markers', 
#     marker = dict(color = eq_df['MW']**2, size = eq_df['MW']**1.5, colorscale = 'Portland')
# )
#data = [trench_data, slab_data, eq_data]
data = [trench_data]

for cont in rot_data:
    data.append(cont)
    
layout = go.Layout(title = 'Trench')
fig = go.Figure(data = data, layout = layout)
fig.update(layout_showlegend=False)
plot(fig)

'temp-plot.html'

In [134]:
print(eq_df['T_AZM'].max(), eq_df['T_AZM'].min())
print(eq_df['P_AZM'].max(), eq_df['P_AZM'].min())
print(eq_df['N_AZM'].max(), eq_df['N_AZM'].min())

360.0 0.0
359.0 0.0
359.0 0.0


In [135]:
print(eq_df['T_PL'].max(), eq_df['T_PL'].min())
print(eq_df['P_PL'].max(), eq_df['P_PL'].min())
print(eq_df['N_PL'].max(), eq_df['N_PL'].min())

89.0 0.0
89.0 0.0
88.0 0.0


In [60]:
test = np.array([[1, 2, 3], [4, 5, 6]])
denom = np.reshape(np.repeat(np.array([1, 2,]), 3), [2, 3])
denom

array([[1, 1, 1],
       [2, 2, 2]])

In [74]:

slab_df.iloc[::5, :]['X']

89656     -345.193324
89661     -369.690746
89666     -394.188238
89671     -418.711932
89676     -443.249254
89681     -467.753875
89686     -492.152781
89691     -516.341696
89696     -540.198163
89701     -563.618575
89706     -586.530251
90697     -345.355625
90702     -369.868901
90707     -394.378446
90712     -418.911494
90717     -443.461549
90722     -467.986939
90727     -492.415109
90732     -516.638082
90737     -540.524563
90742     -563.961084
90747     -586.870159
91736     -335.696593
91741     -360.236194
91746     -384.755315
91751     -409.287249
91756     -433.844259
91761     -458.402030
91766     -482.901576
91771     -507.247377
             ...     
882404   -3198.504562
882409   -3221.884471
882414   -3245.410478
882419   -3269.121415
882424   -3292.954742
883375   -2865.936765
883380   -2890.044594
883385   -2914.124165
883390   -2938.197758
883395   -2962.261303
883400   -2986.284157
883405   -3010.236446
883410   -3034.107651
883419   -3076.812229
883424   -

In [75]:
slab_df.shape

(115512, 6)